I have a requirement in my Thesis where I have to pause a SimPy process until a bit turns True. This notebook has a very simple case to test an approach. 

In [26]:
import simpy
import random

A car that's at a charging station. It charges for 5 minutes and leaves the station when the `charging_status` = 1

In [24]:
def car(env):
    trip_duration = 1
    charging_status = 1
    print("Waiting...")
    yield env.timeout(5)
    #checking for charging_status
    while True:
        print("Waiting for the charging to complete")
        if charging_status == 1:
            yield env.timeout(trip_duration)
        else: 
            pass

In [25]:
env = simpy.Environment()
sample_process = env.process(car(env))
env.run(until = 20)

Waiting for the charging to complete
Waiting for the charging to complete
Waiting for the charging to complete
Waiting for the charging to complete
Waiting for the charging to complete


`charging_status` can be a process

In [73]:
class Car(object):
    def __init__(self, env):
        self.env = env
        self.charging_status = 0
        self.action = self.env.process(self.charging())
    
    def charging(self):
        print("Charging...")
        #yield env.timeout(5)
        print("Waiting for the charging to complete")
        yield self.env.process(self.checking_status)
    
    def checking_status(self): 
        # Checks for the bit in a loop and breaks when the loop
        #yield env.timeout(1)
        self.charging_status = random.randint(0,1)
        print(self.charging_status)
        if self.charging_status ==1:
            yield self.env.timeout(1)

In [74]:
env = simpy.Environment()
car = Car(env)
env.run(until = 20)

Charging...
Waiting for the charging to complete


ValueError: <bound method Car.checking_status of <__main__.Car object at 0x1072fe5c0>> is not a generator.

```
# Remove this workaround if it causes issues in production!
raise ValueError(f'{generator} is not a generator.')
    
    ```

In [79]:
class Car(object):
    def __init__(self, env):
        self.env = env
        # Start the run process everytime an instance is created.
        self.action = env.process(self.run())
    def run(self):
        #while True:
        print('Start parking and charging at %d' % self.env.now)
        charge_duration = 5

        yield self.env.process(self.charge(charge_duration))
        print('Start driving at %d' % self.env.now)
        trip_duration = 2
        yield self.env.timeout(trip_duration)
    
    def waituntil(env, cond_func, delay=1):
        while not cond_func:
            yield env.timeout(delay)

    def charge(self, duration):
        #yield self.env.timeout(duration)
        self.charging_status = random.randint(0,1)
        print("Charging status is " , self.charging_status)
        yield self.waituntil(self.env,self.charging_status)
        #yield env.timeout(1)
    
 

In [80]:
env = simpy.Environment()
car = Car(env)
env.run(until = 20)

Start parking and charging at 0
Charging status is  1


FileNotFoundError: [Errno 2] No such file or directory: '<ipython-input-79-469b36b5f224>'

### Trying out `waituntil`

In [70]:
"""bank14: *waituntil* the Bank door opens"""
from simpy.Simulation import *
from random import expovariate,seed

## Model components ------------------------

door = 'Shut'                                                 
def dooropen():                                               
    return door=='Open'

class Doorman(Process):                                       
    """ Doorman opens the door"""
    def openthedoor(self):
        """ He will open the door when he arrives"""
        global door
        yield hold,self,expovariate(1.0/10.0)                 
        door = 'Open'                                           
        print("%7.4f Doorman: Ladies and "\
              "Gentlemen! You may all enter."%(now(),))

class Source(Process):                                        
    """ Source generates customers randomly"""
    def generate(self,number,rate):       
        for i in range(number):
            c = Customer(name = "Customer%02d"%(i,))
            activate(c,c.visit(timeInBank=12.0))
            yield hold,self,expovariate(rate)

class Customer(Process):                                      
    """ Customer arrives, is served and leaves """
    def visit(self,timeInBank=10):       
        arrive = now()

        if dooropen():
            msg = ' and the door is open.'         
        else:
            msg = ' but the door is shut.'
        print("%7.4f %s: Here I am%s"%(now(),self.name,msg))

        yield waituntil,self,dooropen                         

        print("%7.4f %s: I can  go in!"%(now(),self.name))     
        wait = now()-arrive
        #print "%7.4f %s: Waited %6.3f"%(now(),self.name,wait)

        yield request,self,counter
        tib = expovariate(1.0/timeInBank)
        yield hold,self,tib
        yield release,self,counter

        #print "%7.4f %s: Finished    "%(now(),self.name)

## Experiment data -------------------------

maxTime = 2000.0   # minutes                                     
counter = Resource(1,name="Clerk")

## Model  ----------------------------------

def model(SEED=393939):
    seed(SEED)

    initialize()
    door = 'Shut'
    doorman=Doorman()                                          
    activate(doorman,doorman.openthedoor())                    
    source = Source()                                                         
    activate(source,
             source.generate(number=5,rate=0.1),at=0.0)    
    simulate(until=400.0)

## Experiment  ----------------------------------

model()

ModuleNotFoundError: No module named 'simpy.Simulation'